# Group: EASY5Aplus
## Group member: Ruiting Zhu, Zhedi Zhang, Heng'an Wang, Yuxin Liang, Tao Zhang

In [5]:
import csv
import sys
import scipy.io
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
import time

C:\Users\zhurt\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# Task 1

![3.png](https://s2.loli.net/2024/09/28/hcrXkvLSYuBVbMU.png)
![4.png](https://s2.loli.net/2024/09/28/HspaGuhRZr1qzK6.png)

In [6]:
import numpy as np

def DP(N: int, T: int, alpha: float, pi: float)-> np.ndarray:
    '''
    Use dynamic programming to calculate maximum traded shares at each time period.
    
    N: Total shares
    T: Time periods for trading
    alpha: parameter in calculating actual traded shares
    pi: parameter in calculating actual traded shares
    
    '''

    # initialize variables
    total = np.zeros((T, N + 1))  # initialize total shares traded as a zero matrix
    S = np.zeros((T, N + 1))  # initialize optimal shares plan to trade (not actually traded) as a zero matrix
    M = np.zeros((T, N + 1))  # initialize M as a zero matrix

    iarray = np.arange(N + 1)  
    revi = N - iarray  # remaining stock of shares
    candidates = np.zeros(N + 1)  # candidates of the answer

    # initialize the trade at t=0
    M[0, :] = np.ceil(0.9 * iarray)  # calculate the value of M 
    S[0, :] = iarray  # initial share traded
    total[0, :] = np.ceil((1 - alpha * M[0, :]**pi) * iarray)  # calculate the total revenue traded at time 0

    
    # backward induction
    for t in range(1, T):
        for n in range(0, N + 1):
            # calculate the traded shares of different candidates
            candidates[:n + 1] = total[t - 1, revi[N - n:]] + np.ceil((1 - alpha * np.ceil(0.1 * M[t - 1, revi[N - n:]] + 0.9 * iarray[:n + 1])**pi) * iarray[:n + 1])
            best = np.argmax(candidates[:n + 1])  # find the optimal plan-to-trade amount
            total[t, n] = candidates[:n + 1][best]  # record the optimal actual-traded amount
            S[t, n] = best  # record the optimal plan-to-trade amount
            M[t, n] = np.ceil(0.1 * M[t - 1, int(n - S[t, n])] + 0.9 * S[t, n])  # update M each period using S

    
    # extract optimal trading schedule
    schedule = np.zeros(T)  # build a optimal trading schedule matrix
    remaining_shares = N  

    # backward induction
    for t in range(T - 1, -1, -1):
        schedule[t] = S[t, int(remaining_shares)]  # extract the optimal plan-to-trade amount at t
        remaining_shares -= schedule[t]  # update the remaining shares

    traded = total[T - 1, N]  # total amount traderd

    return schedule, traded


# get result
N, T, alpha, pi = 100000, 10, 0.001, 0.5

t0 = time.time()

schedule, traded = DP(N, T, alpha, pi)

t1 = time.time()

print(f"Parameters: N={N}, T={T}, alpha={alpha}, pi={pi}")
print("Optimal Schedule:")
print(schedule)
print("Total shares traded:")
print(int(traded))
print("Traded percentage of target:")
print(traded / N)
print('Total time DP runs', t1 - t0)


Parameters: N=100000, T=10, alpha=0.001, pi=0.5
Optimal Schedule:
[10100.  9900.  9840.  9888.  9976. 10001. 10048. 10067. 10087. 10093.]
Total shares traded:
90067
Traded percentage of target:
0.90067
Total time DP runs 1094.8082859516144


# Task 2

In [7]:
def data_preprocess(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Use the original file, do basic data cleaning
    '''
    
    # filter needed data
    df = df.iloc[:, : 8]
    column_name = df.loc[2].tolist()
    df = df.loc[3:]
    df.columns = column_name
    df = df.dropna()
    
    # data type conversion
    df.loc[:, 'Dates'] = pd.to_datetime(df['Dates'], format='%m/%d/%y %H:%M')
    df.set_index('Dates', inplace=True)

    columns_to_types = {
    "Open": "float",
    "Close": "float",
    "High": "float",
    "Low": "float",
    "Value": "float",
    "Volume": "int",
    "Number Ticks": "int"}

    for column, type_ in columns_to_types.items():
        df[column] = df[column].astype(type_)
    
    return df

        
def Volume_data(df: pd.DataFrame) -> list:
    '''
    Use the 'Volume' data of Tesla, divide into intervals, present as list and sublist
    '''
    # make sure index is datetime
    df.index = pd.to_datetime(df.index, format='%m/%d/%y %H:%M')

    # (1) Extract the first half of Tesla data, the data should include a whole day range
    # Count total days
    daily_counts = df.groupby(df.index.date).size()
    cumulative_counts = daily_counts.cumsum()
    
    # Find the last complete date in the first half of data
    half_index = len(df) // 2
    last_complete_date = cumulative_counts[cumulative_counts <= half_index].index[-1]

    last_complete_date = pd.Timestamp(last_complete_date)
    first_half_df = df[df.index.normalize() <= last_complete_date]

    # (2) Extract data from 9:30 to 11:30
    filtered_df = first_half_df.between_time('9:30', '11:29')

    # (3) Devide data into intervals, with each interval containing 10 volume data
    result = []
    for date, daily_data in filtered_df.groupby(filtered_df.index.date):
        start_time = datetime.combine(date, datetime.strptime('09:30', '%H:%M').time())
        grouped = []
        for i in range(12):  # there are 12 intervals for each day from 9:30 to 11:30
            start = start_time + timedelta(minutes=10 * i)
            end = start + timedelta(minutes=10)
            group = daily_data[(daily_data.index >= start) & (daily_data.index < end)]['Volume'].tolist()
            grouped.append(group)
        
        grouped = [group for group in grouped if group]
        result.extend(grouped)

    return result



def get_score(ori_data: pd.DataFrame, optimal: list) -> float:
    '''
    Compare Tesla Volume with schedule we get in Q1, get final score
    ''' 
    # Tesla Volume data
    precessed_data = data_preprocess(ori_data)
    Tesla_volume = Volume_data(precessed_data)
    
    # optimal schedule of Q1
    #optimal_schedule = [int(x) for x in optimal]
    
    # interval_score
    interval_scores = []
    for interval in Tesla_volume:
        interval_score = sum(min(x, (y / 100)) for x,y in zip(optimal, interval))
        interval_scores.append(interval_score)

    # final_score
    final_score = sum(interval_scores) / len(interval_scores)

    return final_score

In [61]:
# get result
Tesla_ori_data = pd.read_csv("TSLA.csv")
optimal = [10100, 9900, 9840, 9888, 9976, 10001, 10048, 10067, 10087, 10093]
get_score(Tesla_ori_data, schedule)

<ipython-input-61-828282e05be0>:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,10,11,12,13,14,15,16,19,20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  Tesla_ori_data = pd.read_csv("TSLA.csv")


8113.97704545455

# TASK 3

### Method 1: 

It takes quite a long time and didn't finished before the ddl.

In [62]:
def best_pi(start: float, end: float, step: float) -> np.ndarray:
    '''
    Find the optimal pi given N, T and alpha to maximize the score.
    This function allow self-defined step of pi.
    '''
    # create an array containing all pi values
    pi_values = np.arange(start, end + step, step)

    # Calculate score for each pi
    scores = np.array([get_score(Tesla_ori_data, DP(N, T, alpha, pi)[0]) for pi in pi_values])

    # find index, and optimal pi
    best_index = np.argmax(scores)
    best_pi_value = pi_values[best_index]
    max_score = scores[best_index]

    return best_pi_value, max_score


In [ ]:
N, T, alpha = 100000, 10, 0.001
start, end = 0.3, 0.7

print('Given N = 100000, T = 10, alpha = 0.001')
for i in [0.1, 0.01, 0.001]:
    optimal_pi, max_score = best_pi(start, end, i)
    print(f'When step = {i}, best pi ={optimal_pi}, maximum score ={max_score} ')


Given N = 100000, T = 10, alpha = 0.001
When step = 0.1, best pi =0.4, maximum score =8114.817954545459 


### Method 2: 

It takes relatively shorter time but didn't get enough time to finished before the ddl. Here we use N=10000 to see the effect.

In [10]:
# define loss function
def loss_function(pi, Tesla_ori_data, N, T, alpha) -> float:
    '''
    define the loss function according to the score
    '''
    optimal, traded = DP(N, T, alpha, pi)
    score = get_score(Tesla_ori_data, optimal)
    return -score*1000


# define gradient
def numerical_gradient(pi, Tesla_ori_data, N, T, alpha, epsilon=1e-3) -> float:
    '''
    calculate gradient using finite difference method
    '''
    loss_plus_epsilon = loss_function(pi + epsilon, Tesla_ori_data, N, T, alpha)
    loss_at_pi = loss_function(pi, Tesla_ori_data, N, T, alpha)
    grad = (loss_plus_epsilon - loss_at_pi) / epsilon
    return grad


# find the optimal pi between 0.3 and 0.7
def gradient_descent_pi(Tesla_ori_data, N, T, alpha, start_pi=0.3, learning_rate=0.001, tolerance=1e-6, max_iterations=1000) -> float:
    '''
    find the optimal pi that can reach the bottom of the gradient
    self-define start_pi, learning_rate, tolerance according to the learning outcome
    '''
    pi = start_pi  # initialize pi
    for iteration in range(max_iterations):
        grad = numerical_gradient(pi, Tesla_ori_data, N, T, alpha)  # calculate gradient
        new_pi = pi - learning_rate * grad  

        # ensure the range of pi
        new_pi = np.clip(new_pi, 0.3, 0.7)

        # print iteration
        print(f"Iteration {iteration + 1}: pi = {pi}, grad = {grad}, loss = {loss_function(pi, Tesla_ori_data, N, T, alpha)}")

        # check tolerance
        if abs(new_pi - pi) < tolerance:
            print("Convergence achieved.")
            break

        pi = new_pi  

    # optimal pi
    return pi


N = 10000
T = 10
alpha = 0.001

best_pi = gradient_descent_pi(Tesla_ori_data, N, T, alpha)
print(f"Optimal pi: {best_pi}")
print(f"Maximum score: {-loss_function(best_pi, Tesla_ori_data, N, T, alpha)/1000}")


Iteration 1: pi = 0.3, grad = -81033510.10101661, loss = -6278553.724747476
Iteration 2: pi = 0.7, grad = -2187613.6363632977, loss = -6367240.631313137
Convergence achieved.
Optimal pi: 0.7
Maximum score: 6367.240631313137


# Individual Contribution:

* Ruiting Zhu: 
  * Responsible for solving all questions, while adding explanations and code comments for all. 
  * Deduct the mathematical model of the question 1, figure out the logic, resolve member's questions and discuss with the group, jointly write and refine the code. 
  * Completed most of the work of question 2 including code writing and briefness checking. 
  * Solved question 3 independently using two approaches with briefness checking and runtime optimization. 
  * Organized and participated in all group discussions, discussed the problem-solving approach, and joined the TA's office hours with teammates.
* Zhedi Zhang: 
  * Write and improve the code of question 1. Deduct the mathematic model with teammates. Write the final version of the code. 
  * Discussion: join all the discussion parts and go to the ta’s office hour asking questions.
* Heng'an Wang: 
  * Participate in all zoom meeting group discussion and provide the draft code version of task1 and the run result of optimal schedule for other team members' reference.
  * Discuss the logic of dynamic programming with other team members and apply the DP1-baseline approach model from lecture slide to task1 to figure out the Vectorization and forloop part of the code.
  * Debug task1 and task3 code in v1 to test and optimize the run time.
  * Join with other team members in TA's office hour to ask questions and discuss how to correct forloop and update M in different T.
* Kexin Liang: 
  * Write and Improve Code. Provide data cleaning code for question 2 and improve the calculation speed for the third question.
  * Cooperation and discussion. Specifically discussed the project’s tasks logic with others，the project’s method used and shared own code with others. Attend in person discussion.
* Tao Zhang: **(highly-conflicted contribution, need further discussion with professor or TA)**
  * Discussion and determination of the code approach for Task 1 & Task 2; Writing the code for Task 1 (non-adopted version)；Functional testing and validation of Task 3 in version 1; Functional testing and validation of Task 1, 2, & 3 in version 2.